In [4]:
import pandas as pd
import json
import os
import requests
from bs4 import BeautifulSoup  # you need to install this package, use: pip install beautifulsoup4

def load_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    tweets = []
    for item in data:
        tweet_id = item['Tweet_ID']
        stock_ticker = item['Target_Ticker']
        sentiment = item['Sentiment']
        tweets.append({'tweet_id': tweet_id, 'stock_ticker': stock_ticker, 'sentiment': sentiment})
    return pd.DataFrame(tweets)

def get_tweet_text(tweet_id):
    try:
        url = f'https://twitter.com/anyuser/status/{tweet_id}'
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            tweet_content = soup.find('div', {'class': 'css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0'}).text
            return tweet_content
        else:
            return None
    except:
        return None

# Now, load the data
train_data = load_data('TweetFinSent_Train.json')
test_data = load_data('TweetFinSent_Test.json')

# And use the function to get tweet texts
train_data['tweet_text'] = train_data['tweet_id'].apply(get_tweet_text)
test_data['tweet_text'] = test_data['tweet_id'].apply(get_tweet_text)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

X_train, X_test, y_train, y_test = train_test_split(train_data['tweet_text'], train_data['sentiment'], test_size=0.2, random_state=42)

model = make_pipeline(TfidfVectorizer(), LogisticRegression())
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print(f'Model accuracy: {accuracy:.2f}')


KeyError: 'tweet_text'